In [1]:
import sympy as sp
import numpy as np
#import math

#### variables globales
1. variables : une liste contient des symboles créés; utilisé dans la partie expressions
2. dictionaire_auxiliary_variable : des variables créées pour remplacer le produit de deux variables dans p et q
3. res : number , pour créer le nom des variables
4. variables_Ising : variables dans Hamiltonain expresstion
5. dict_variables_binary : map variables dans hamiltonian et variables dans modèle

In [2]:
variables=[]
dictionaire_auxiliary_variable={}
res=1
variables_Ising =[]
dict_variable_model_map={}

# Expression

## Step 0 : dénifir la fonction f() = (N-pq)²

In [3]:
# Créer un symbole à partir d'un nom en string
def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

### Définir variables pour p et q
##### p en binaire peut être représenté comme:
##### (pl1-1,..p1,1) = pl1-1 * 2 ** (l1-1) + ... + p1 * 2 **1 +1
##### q en binaire peut être représenté comme:
##### (ql2-1,..,q1,1) = ql2-1 * 2 ** (l2-1) + ... + q1 * 2 **1 +1

In [4]:
def set_variable(l1,l2):
    #p
    expr_p = 1
    for i in range(1,l1):
        var_name = ('p'+ str(i))
        var = create_variables(var_name)
        expr_p = 2**i * var + expr_p 
    #q
    expr_q = 1
    for i in range(1,l2):
        var_name = ('q'+ str(i))
        var = create_variables(var_name)
        expr_q = 2**i * var + expr_q
    return (expr_p,expr_q)

### Expression = (N-p*q)²
##### Le min de cette expression dans Z est 0, dans ce cas N = p*q.

In [5]:
def define_function(N,l1,l2):
    expr_pq = 1
    pq_variable = set_variable(l1,l2)
    for item in pq_variable:
        expr_pq = expr_pq * item
    return (N - expr_pq)**2

#expr = define_function(15,2,3)
#expr

## Step 1: Réduire la fonction (le power de toutes les variables en 1)
##### p_1**2 = p_1, car 1 * 1 = 1 et 0 * 0 = 0

In [6]:
def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

#expr_reduct_power = reduct_variable_power_function(expr)
#expr_reduct_power


## Step 2: Réduire les termes pour que ils ont tous une degrée <= 2

In [7]:
# les varaiables sont sort par la lettre puis un nombre: "x12" -> "x" 12
def sort_key(variable):
    name = str(variable)
    # Split le nom du variable aux deux parties: letter & number 
    letter = name[0]
    number = int(name[1:])
    return (letter, number)

### réduction degrée formule

In [8]:
def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term

# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):    
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]
    
    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1 
    else:
        x_4 = key 

    expr_reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = rest_expr * expr_reduct_formula
    return sp.expand(expr_reduct)


### Réduction procède

In [9]:
def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :  
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste) 
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction: # récursive
        return reduct_demension_expression(new_expr)
    else:
        return new_expr

#reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
#reduct_demension_expr

# Trouver paramètres pour la construction de modèle Ising / Qubo 

## Step3: Calculer h et J

1. Ising : transfer de Qubo à Ising 

De reduct_demension_expr, on utilise les formule comme:

    variable = (variable_ising +1)/2

In [10]:
# créer variable à partir des variables pour modèle Ising
def create_variable_Ising(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables_Ising.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol


# définir H
def transfers_qubo2Ising(reduct_demension_expr):
    n = len(variables)
    H = 0

    # Pour chaque variable, on crée une variables d'Ising avec le nom "si"
    for i in range(1,n+1):
        var_name = ('s'+ str(i))
        var_model = create_variable_Ising(var_name)
        dict_variable_model_map[variables[i-1]] = var_model

    # variable = (variable_ising +1)/2
    # subs_relations = [(variables[i], (variables_Ising[i] + 1) / 2) for i in range(n)]
    subs_relations = [(variables[i], (dict_variable_model_map[variables[i]] + 1) / 2) for i in range(n)]
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

#expr_ising = transfers_qubo2Ising(reduct_demension_expr)
#expr_ising


In [11]:
def get_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None

In [12]:
def build_h(expr):
    #len de h est le nombre de variables 
    h=()
    for term in expr.args:
        if len(term.free_symbols) == 1:
            h = h + (term.as_coeff_mul()[0]/-2 ,)
    return h


def build_J_h(expr):

    terms = expr.args 

    # 1-variable : reconstruire une expression pour calculer le coef des termes
    new_expr = 0

    matrix_J = np.zeros((len(variables), len(variables)), dtype=int)

    for term in terms:
        if len(term.free_symbols) == 2:
            # 2-variables : créer la matrice J
            list_variable_term = sorted(term.free_symbols, key=sort_key)
            coef = term.as_coeff_mul()[0]
            i = variables.index(get_key_by_value(dict_variable_model_map, list_variable_term[0]))
            j = variables.index(get_key_by_value(dict_variable_model_map, list_variable_term[1]))
            
            matrix_J[i][j] = coef/2
        else:
            # 1-variable
            new_expr += term

    tuple_h = build_h(new_expr)
    return tuple_h,matrix_J

In [13]:
def build_h_Q(expr):
    #len de h est le nombre de variables 
    h=()
    for term in expr.args:
        if len(term.free_symbols) == 1:
            h = h + (term.as_coeff_mul()[0],)
    return h


def build_J_h_Q(expr):

    terms = expr.args 

    # 1-variable : reconstruire une expression pour calculer le coef des termes
    new_expr = 0

    matrix_J = np.zeros((len(variables), len(variables)), dtype=int)

    for term in terms:
        if len(term.free_symbols) == 2:
            # 2-variables : créer la matrice J
            list_variable_term = sorted(term.free_symbols, key=sort_key)
            coef = term.as_coeff_mul()[0]     
            i = variables.index(list_variable_term[0])
            j = variables.index(list_variable_term[1])
            
            matrix_J[i][j] = coef
        else:
            # 1-variable
            new_expr += term

    tuple_h = build_h_Q(new_expr)
    return tuple_h,matrix_J

sum up fonction

In [14]:
# obtient l'expression finale
def get_reduct_expression(N,l1,l2):
    expr = define_function(N,l1,l2)
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_demension_expr = reduct_demension_expression(expr_reduct_power)
    return reduct_demension_expr


#1. Ising
def get_h_and_J_Ising(N,l1,l2):
    reduct_demension_expr = get_reduct_expression(N,l1,l2)
    ising_expr = transfers_qubo2Ising(reduct_demension_expr) #trans
    tuple_h,matrix_J = build_J_h(ising_expr)
    return tuple_h,matrix_J

#2. Qubo
def get_h_and_J_Qubo(N,l1,l2):
    reduct_demension_expr = get_reduct_expression(N,l1,l2)
    tuple_h,matrix_J = build_J_h_Q(reduct_demension_expr)
    return tuple_h,matrix_J

# Modèle Ising/Qubo 

In [15]:
#import qubo modèle huristique
from pyqubo import Spin,Binary
from pyqubo import solve_qubo
import neal

## Step4: Définir la formule Hamiltonian

1. Ising

In [16]:
# définir formule H
def H_pyqubo_ising(N,l1,l2):
    (h,J)=get_h_and_J_Ising(N,l1,l2)
    print("****************")
    print(f"N est : {N}")
    print(f"h est : {h}")
    print(f"J est : {J}")

    hamiltonian_expr = 0
    n=len(h)
    spins=[]

    for i in range(1,n+1):
        var_name = ('s'+ str(i))
        spin = Spin(var_name)
        spins.append(spin)
        dict_variable_model_map[variables[i-1]] = var_name

    for i in range(n):
        for j in range(n):
            hamiltonian_expr -= J[i, j] * spins[i] * spins[j]
        hamiltonian_expr -= h[i] * spins[i]
    return hamiltonian_expr

2. Qubo

In [17]:
# not clean yet

def H_pyqubo_qubo(N,l1,l2):
    (h,J)=get_h_and_J_Qubo(N,l1,l2)
    print("****************")
    print(f"N est : {N}")
    print(f"h est : {h}")
    print(f"J est : {J}")

    hamiltonian_expr = 0
    n=len(h)
    binarys=[]

    for i in range(1,n+1):
        var_name = ('s'+ str(i))
        binary = Binary(var_name)
        binarys.append(binary)
        dict_variable_model_map[variables[i-1]] = var_name

    for i in range(n):
        for j in range(n):
            hamiltonian_expr -= J[i, j] * binarys[i] * binarys[j]
        hamiltonian_expr -= h[i] * binarys[i]
    return hamiltonian_expr


### Compiler pour obtenir modèle

1. qubo, offset = model.to_qubo() to get QUBO coefficients
2. linear, quadratic, offset = model.to_ising() to get Ising coefficients

Comme on voudrais obtenir des valeurs \in {0,1}, on utilise to_qubo()

In [18]:
def get_solution(hamiltonian_expr):
    model = hamiltonian_expr.compile()
    bqm = model.to_bqm()
    sa = neal.SimulatedAnnealingSampler()
    sampleset = sa.sample(bqm, num_reads=1000)
    decoded_samples = model.decode_sampleset(sampleset)
    best_sample = min(decoded_samples, key=lambda x: x.energy)
    return best_sample.sample

In [19]:
# not clean
def model_qubo(hamiltonian_expr):
    #Compile the Hamiltonian to get a model
    model = hamiltonian_expr.compile()  

    qubo, offset = model.to_qubo()
    solution = solve_qubo(qubo, num_reads=1000)
    return solution

In [20]:
# not clean
def model_ising(hamiltonian_expr):
    # Compile the Hamiltonian to get a model
    model = hamiltonian_expr.compile()  
    
    bqm = model.to_bqm()
    sa = neal.SimulatedAnnealingSampler()
    sampleset = sa.sample(bqm, num_reads=1000)
    decoded_samples = model.decode_sampleset(sampleset)
    best_sample = min(decoded_samples, key=lambda x: x.energy)
    return best_sample.sample

# Call function

In [21]:
def print_p_q(solution):
    p_bits = []
    q_bits = []

    print(solution)
    
    for var in variables:
        if var.name.startswith('p'):
            p_bits.append(solution[dict_variable_model_map[var]])
        elif var.name.startswith('q'):
            q_bits.append(solution[dict_variable_model_map[var]])

    p_bits.append(1)
    q_bits.append(1) 

    p_bin_str = ''.join(map(str, p_bits))
    q_bin_str = ''.join(map(str, q_bits))

    p_dec = int(p_bin_str, 2)
    q_dec = int(q_bin_str, 2)

    return (p_dec , q_dec)


def factorization_I(N,l1,l2):
    global variables, variables_Ising, dictionaire_auxiliary_variable, res
    variables = []
    variables_Ising = []
    dictionaire_auxiliary_variable = {}
    res = 1

    hamiltonian_expr_i = H_pyqubo_ising(N,l1,l2)
    solution = get_solution(hamiltonian_expr_i)
    (p,q) = print_p_q(solution)
    print(f"La factorisation de {N} est {p} * {q}.")
    return p*q == N

def factorization_Q(N,l1,l2):
    global variables, variables_Ising, dictionaire_auxiliary_variable, res
    variables = []
    variables_Ising = []
    dictionaire_auxiliary_variable = {}
    res = 1

    hamiltonian_expr_q = H_pyqubo_qubo(N,l1,l2)
    solution = get_solution(hamiltonian_expr_q)
    (p,q) = print_p_q(solution)
    print(f"La factorisation de {N} est {p} * {q}.")
    return p*q == N


In [22]:
N=15
l1=2
l2=3
factorization_I(N,l1,l2)
factorization_Q(N,l1,l2)

****************
N est : 15
h est : (58, 50, 12, -80)
J est : [[  0  25  -6 -64]
 [  0   0   2 -64]
 [  0   0   0  16]
 [  0   0   0   0]]
{'s1': 1, 's2': 1, 's3': 0, 's4': 0}
La factorisation de 15 est 3 * 5.
****************
N est : 15
h est : (-96, -52, -52, 768)
J est : [[   0  200  -48 -512]
 [   0    0   16 -512]
 [   0    0    0  128]
 [   0    0    0    0]]
{'s1': 0, 's2': 0, 's3': 1, 's4': 1}
La factorisation de 15 est 1 * 3.


False

In [23]:
N=21
l1=2
l2=3
factorization_I(N,l1,l2)
factorization_Q(N,l1,l2)

****************
N est : 21
h est : (82, 62, 36, -80)
J est : [[  0  19 -18 -64]
 [  0   0   2 -64]
 [  0   0   0  16]
 [  0   0   0   0]]


{'s1': 1, 's2': 1, 's3': 1, 's4': 0}
La factorisation de 21 est 3 * 7.
****************
N est : 21
h est : (-144, -76, -76, 768)
J est : [[   0  152 -144 -512]
 [   0    0   16 -512]
 [   0    0    0  128]
 [   0    0    0    0]]
{'s1': 0, 's2': 0, 's3': 1, 's4': 1}
La factorisation de 21 est 1 * 3.


False

In [24]:
N=33
l1=2
l2=4
factorization_I(N,l1,l2)
factorization_Q(N,l1,l2)

****************
N est : 33
h est : (374, 204, 146, 56, -240, -320)
J est : [[   0   71   86  -52 -192 -256]
 [   0    0    2    4 -192    0]
 [   0    0    0    8   16 -256]
 [   0    0    0    0   32   64]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]]
{'s1': 1, 's2': 1, 's3': 1, 's4': 0, 's5': 0, 's6': 0}
La factorisation de 33 est 3 * 13.
****************
N est : 33
h est : (-448, -240, -124, -124, 2304, 3072)
J est : [[    0   568   688  -416 -1536 -2048]
 [    0     0    16    32 -1536     0]
 [    0     0     0    64   128 -2048]
 [    0     0     0     0   256   512]
 [    0     0     0     0     0     0]
 [    0     0     0     0     0     0]]
{'s1': 0, 's2': 0, 's3': 0, 's4': 1, 's5': 1, 's6': 1}
La factorisation de 33 est 1 * 3.


False

In [25]:
N=35
l1=3
l2=3
factorization_I(N,l1,l2)
factorization_Q(N,l1,l2)

****************
N est : 35
h est : (640, 640, 280, 272, 136, -80, -160, -240, -608, -640)
J est : [[   0  258    5 -302 -512  -64    0  512    0    0]
 [   0    0   50 -324 -512    0 -192    0  512    0]
 [   0    0    0   66   16  -64 -192    0    0 -128]
 [   0    0    0    0  304   16   48  512  512 -128]
 [   0    0    0    0    0    0    0 -128 -128   32]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]]
{'s1': 1, 's3': 1, 's10': 0, 's2': 1, 's4': 0, 's5': 0, 's6': 0, 's7': 0, 's8': 0, 's9': 0}
La factorisation de 35 est 7 * 5.
****************
N est : 35
h est : (-6144, -6144, -256, -256, -132, -132, 768, 1536, 2304, 6144)
J est : [[    0  2064    40 -2416 -4096  -512     0  4096     0     0]
 [    0     0   400 -2592 -4096     0 -1536     0  4096     0]
 [    0     

False

In [26]:
N=39
l1=2
l2=4
factorization_I(N,l1,l2)
factorization_Q(N,l1,l2)

****************
N est : 39
h est : (422, 228, 158, 104, -240, -320)
J est : [[   0   65   74  -76 -192 -256]
 [   0    0    2    4 -192    0]
 [   0    0    0    8   16 -256]
 [   0    0    0    0   32   64]
 [   0    0    0    0    0    0]
 [   0    0    0    0    0    0]]
{'s1': 1, 's2': 1, 's3': 1, 's4': 0, 's5': 0, 's6': 0}
La factorisation de 39 est 3 * 13.
****************
N est : 39
h est : (-544, -288, -148, -148, 2304, 3072)
J est : [[    0   520   592  -608 -1536 -2048]
 [    0     0    16    32 -1536     0]
 [    0     0     0    64   128 -2048]
 [    0     0     0     0   256   512]
 [    0     0     0     0     0     0]
 [    0     0     0     0     0     0]]
{'s1': 0, 's2': 0, 's3': 0, 's4': 1, 's5': 1, 's6': 1}
La factorisation de 39 est 1 * 3.


False